# Paraglider flight mechanics in  2d

  $(m_1,x_1)$ pilot
  
  $(m_2,x_2)$ wing
  
  $F_x$, $F_y$, aerodynamic force in the rest system

In [ ]:
def make_fig(x1,y1,x2,y2,CM=True,m1=1,m2=1):
    x1 = x1 - (m1*x1+m2*x2)/2
    y1 = y1 - (m1*y1+m2*y2)/2
    plts = []
    plts +=[ point((x1,y1),color='red')]
    plts +=[ point((x2,y2),color='brown')]
    plts +=[ line ([(x1,y1),(x2,y2)])]
    plts += [ellipse((x2,y2),1,.4,arctan2(y1-y2,x1-x2)-pi/2,fill=True,alpha=0.3)]
    plt = sum(plts)
    
    return plt

In [ ]:
plt = make_fig(1,2,3,4)
plt.show(figsize=4,aspect_ratio=1,axes=false)

In [ ]:
import numpy as np
var('t')
var('l g Fx Fy m1 m2')
xy_wsp = [('x1','x1'),('y1','y1'),('x2','x2'),('y2','y2')]

uv_wsp = [('phi','\phi'),('x','x'),('y','y')]

for v,lv in uv_wsp+xy_wsp:
    var("%s"%v,latex_name=r'%s'%lv)
    vars()[v.capitalize()] = function(v.capitalize())(t)
    var("%sdd"%v,latex_name=r'\ddot %s'%lv)
    var("%sd"%v,latex_name=r'\dot %s'%lv)
    var("d%s"%v,latex_name=r'\delta %s'%lv)

uv = [vars()[v] for v,lv in uv_wsp]

xy = [vars()[v] for v,lv in xy_wsp]
to_fun=dict()
for v,lv in uv_wsp:
    to_fun[vars()[v]]=vars()[v.capitalize()]
    to_fun[vars()[v+"d"]]=vars()[v.capitalize()].diff()
    to_fun[vars()[v+"dd"]]=vars()[v.capitalize()].diff(2)
to_var = dict((v,k) for k,v in to_fun.items())

x2u = {x2:x,y2:y, x1:x+l*sin(phi),y1:y-l*cos(phi)}

for w in xy:
    vars()['d'+repr(w)+'_polar']=sum([w.subs(x2u).diff(w2)*vars()['d'+repr(w2)] for w2 in uv])
    #show(vars()['d'+repr(w)+'_polar'])

dAlemb = (m1*x1.subs(x2u).subs(to_fun).diff(t,2))*dx1_polar + \
         (m1*y1.subs(x2u).subs(to_fun).diff(t,2)+m1*g)*dy1_polar+\
         (m2*x2.subs(x2u).subs(to_fun).diff(t,2)-Fx)*dx2_polar + \
         (m2*y2.subs(x2u).subs(to_fun).diff(t,2)+m2*g-Fy)*dy2_polar
dAlemb = dAlemb.subs(to_var)

In [ ]:
show(dAlemb)

In [ ]:
rown=[]
for v in uv:
    rown.append(dAlemb.expand().coefficient(vars()['d'+repr(v)]).trig_simplify())
drugie =   [vars()[repr(v)+'dd'] for v in uv]
table(rown)


In [ ]:
#drugie =   [vars()[repr(v)+'dd'] for v in uv]
sol = solve( rown,drugie)
show( map(lambda x:x.trig_simplify(),sol[0]) )

In [ ]:
rhs = map(lambda x:x.rhs().trig_simplify(),sol[0])
show(rhs)

In [ ]:
Cx(x) = .2+sin(x)^2
Cz(x) = 5*sin(x*2)
plot(Cx(x),(x,-pi,pi),figsize=5)+\
 plot(Cz(x),(x,-pi,pi),color='red')+\
 plot(Cz(x)/Cx(x),(x,-pi,pi),color='green')

In [ ]:
#Cx = lambda x:1.0
#Cz = lambda x:1.0
trim = -0.24
alpha = arctan2(yd,xd)
forces={Fx:(-Cx(phi-alpha+trim)*cos(alpha) - Cz(phi-alpha+trim)*sin(alpha) )*(xd^2+yd^2),Fy:(-Cx(phi-alpha+trim)*sin(alpha) + Cz(phi-alpha+trim)*cos(alpha))*(xd^2+yd^2)}
#show( map(lambda x:x.subs(forces),rhs) )
dof3 = [phid,xd,yd] + map(lambda x:x.subs(forces),rhs)

In [ ]:
%%time
params = {m1:1.3,m2:2,l:1}
ode = map(lambda x:x.subs(params), dof3) 
times = srange(0,8,0.1)
numsol = desolve_odeint(ode,[0,0,0,.0,2.21,4], \
                        times, [phi,x,y,phid,xd,yd])

In [ ]:
p = line(zip(numsol[:,1],numsol[:,2]),figsize=9)#,marker='o')
p += line(zip(numsol[:,1]+np.sin(numsol[:,0]),numsol[:,2]-np.cos(numsol[:,0])),color='red',aspect_ratio=1)#,marker='o')
p.show()

In [ ]:
line(zip(times,np.sqrt(numsol[:,4]**2+numsol[:,5]**2)) )

In [ ]:
line(zip(times,numsol[:,5]) )

In [ ]:
every = max(int(numsol.shape[0]/25),1)
every

In [ ]:
def plot_system(data,l=1):
    phi,x,y = data[0:3]
    w,vx,vy = data[3:]
    x2,y2 = x+l*sin(phi),y-l*cos(phi)
    p =  point((x,y),size=40,color='blue',figsize=4)
    p += point((x2,y2),size=40,color='red')
    p += line([(x,y),(x2,y2)])
    v = vector([vx,vy])
    if v.norm()>0:
        v = v/v.norm()
        p += arrow((x,y),(x+v[0],y+v[1]),aspect_ratio=1)
    return p

In [ ]:
plts = [p+plot_system(data,l=1) for data in numsol[::every,:]]

In [ ]:
%%time
anim = animate(plts,xmax=125)
anim.show()

In [ ]:
plot_system(data,l=1)

## $C_L$ from data

In [ ]:
data = [(x,2*x^3+random()*0.61) for x in srange(0.,10,0.41)]
plt = point(data)
var('c b')
model(x) = c*x^3 + b
fit = find_fit( data, model, solution_dict=True)
plt_m = plot(model(x).subs(fit),(x,-10,20))
(plt_m+plt).show(figsize=5)
model(x).subs(fit)

\newpage